In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import sys
import json
from tqdm.auto import tqdm
from collections import Counter
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
file_path = "/content/drive/My Drive/한양대학교/브런치 사용자를 위한 글 추천 대회/data/"
#file_path = input("file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/data)>>")

In [3]:
class Doc2VecCorpus:
    def __init__(self, fname):
        self.fname = fname    
    def __iter__(self):
        for num in range(7):
            for line in open(self.fname + '.' + str(num), 'r'):
                tmp = json.loads(line)
                yield TaggedDocument(
                    words = [j for i in tmp["morphs"] for j in i], 
                    tags = [tmp["id"]])

doc2vec_corpus = Doc2VecCorpus(file_path+"data")

In [4]:
doc2vec_model = Doc2Vec(doc2vec_corpus, vector_size=300)

In [5]:
doc2vec_model.save(file_path+"model.doc2vec")

In [6]:
doc2vec_model = Doc2Vec.load(file_path+"model.doc2vec")

In [7]:
read_data = pd.read_pickle(file_path+"read.pkl")

read_data["tmp_file_name"] = read_data["file_name"].apply(lambda x : int(x[:8]))

dev_users_data = []

with open(file_path+"predict/dev.users", 'r') as f:
    for line in f:
        line = line.strip()
        dev_users_data.append(line)

dev_users_data = pd.DataFrame({"user_id":dev_users_data})
dev_users_data['recommend'] = ''

read_dev_users_data = pd.merge(read_data, dev_users_data, left_on='user_id', right_on='user_id', how='inner')
read_dev_users_data = read_dev_users_data.groupby("user_id").apply(lambda x : [element for array in x["content_id"].values for element in array])
read_dev_users_data = pd.DataFrame(read_dev_users_data).reset_index()
read_dev_users_data.columns = ['user_id','read_id']

print(read_dev_users_data)

                                user_id                                            read_id
0     #000fe61478d384d09f3bcdd0c2f5227d  [@ddamimovie_1485, @ddamimovie_1485, @ddamimov...
1     #0029e554ab774e79b69ee9217dee06db  [@binkond_1183, @readream3_25, @mothertive_43,...
2     #005f71b15b9423f335b85626ceec86a6  [@minigorae_248, @minigorae_250, @jiholim_82, ...
3     #0085e940b6e94be6efe4e275d1966615  [@sssfriend_52, @gracefull_22, @prestigegorill...
4     #008d4d3ae2f49a95f9d82d9e1d8a97ac  [@klida8866_32, @klida8866_32, @klida8866_33, ...
...                                 ...                                                ...
2995  #ffab618a20ff46fdac7bb842b28f005d  [@holidaymemories_139, @jordan777_2515, @jorda...
2996  #ffae495f7c65c0ab7cb4eac72e84bc20  [@the8work_78, @the8work_78, @the8work_80, @th...
2997  #ffb3a1e1514e9afa92914cc1a74e4ce6  [@babytone2_14, @lucete0710_3, @12myapril_149,...
2998  #ffeabb5aaf948bcc15be2f88ebeab3c4  [@moonwalker_7, @moonwalker_7, @moonwalker_7, ...

In [8]:
read_data = pd.read_pickle(file_path+"read.pkl")
content = Counter([j for i in read_data["content_id"].tolist() for j in i])

In [9]:
def doc2vec_most_similarity(x):
    try:
        tmp = []
        for i in doc2vec_model.docvecs.most_similar(x):
            try:
                view = content[i[0]]
            except:
                view = 0
            tmp.append((i[0],i[1],view))
        return tmp
    except:
        return list()

read_id = pd.DataFrame({"id":list(set([j for i in read_dev_users_data['read_id'].values for j in i]))})
read_id['most_similarity'] = read_id['id'].apply(lambda x : doc2vec_most_similarity(x))
read_id.to_pickle('/content/drive/MyDrive/한양대학교/브런치 사용자를 위한 글 추천 대회/data/document_most_similarity.pkl')

print(read_id)

                    id                                    most_similarity
0          @pdazim_103  [(@seamansfilm_2, 0.7558492422103882, 0), (@ji...
1        @dailydraw_62  [(@khbang21_554, 0.8451170325279236, 1), (@mol...
2          @mijople_29  [(@ddamimovie_1429, 0.7502186298370361, 308), ...
3        @milhouse22_2  [(@4nab2_13, 0.8038478493690491, 13), (@mistyr...
4       @wikitree_2794  [(@soul98_126, 0.763897180557251, 45), (@bong_...
...                ...                                                ...
191589    @savvyoon_83  [(@paxcom_121, 0.7521566152572632, 0), (@hitch...
191590    @jyyoonmd_12  [(@jyyoonmd_85, 0.7064839005470276, 0), (@jyyo...
191591      @sonst7_97  [(@sonst7_78, 0.6962778568267822, 28), (@sonst...
191592    @byminigun_1  [(@yoa2860_310, 0.6594705581665039, 2), (@gidm...
191593  @expediakr_398  [(@expediakr_434, 0.7074766159057617, 0), (@je...

[191594 rows x 2 columns]


In [10]:
read_id = pd.read_pickle('/content/drive/MyDrive/한양대학교/브런치 사용자를 위한 글 추천 대회/data/document_most_similarity.pkl')

In [11]:
popular = pd.read_csv('/content/drive/MyDrive/한양대학교/브런치 사용자를 위한 글 추천 대회/brunch-article-recommendation-master/recommend.txt', sep = " ", header=None).iloc[0,1:].tolist()

In [12]:
for x in tqdm(range(len(read_dev_users_data))):
    total = []
    for i in set(read_dev_users_data.iloc[x,1]):
        try:
            total.extend(read_id[read_id['id']==i]['most_similarity'].tolist()[0])
        except:
            pass
    total = sorted(total, key = lambda x : x[1], reverse=True)
    if len(total) == 0:
        total = popular
    elif len(total) >= 100:
        total = list(set([i[0] for i in sorted(total[:100], key = lambda x : x[2], reverse=True)]))
        for i in popular:
            if len(total) == 100:
                break
            if i not in total:
                total.append(i)
    elif len(total) < 100:
        total = list(set([i[0] for i in sorted(total, key = lambda x : x[2], reverse=True)]))
        for i in popular:
            if len(total) == 100:
                break
            if i not in total:
                total.append(i)
    dev_users_data.loc[dev_users_data.user_id == read_dev_users_data.iloc[x,0], 'recommend'] = [total]

dev_users_data.to_pickle('/content/drive/MyDrive/한양대학교/브런치 사용자를 위한 글 추천 대회/data/dev_users_data.pkl')

print(dev_users_data)


                                user_id                                          recommend
0     #d6866a498157771069fdf15361cb012b  [@volo_309, @hosslee_367, @honeytip_325, @hoss...
1     #f963fb8c5d9d14d503fc4e80bd8617b4  [@dailylife_680, @perytail_370, @youngjishin_1...
2     #87a6479c91e4276374378f1d28eb307c  [@cysstory_108, @woochul4532_1, @tnrud572_70, ...
3     #677e984e245b344f61dc5d3cc1f352c8  [@kyoonhopark_121, @coloresprit_5, @jmg5308_77...
4     #519f45eb14e4807e8714fb7e835463eb  [@tenbody_1590, @mingamsungfilm_34, @tenbody_1...
...                                 ...                                                ...
2995  #c8bdf5ee13c87360dd14ffea7f63483f  [@forhappywomen_14, @kayyoon_19, @kimarang_3, ...
2996  #e8dbed7a4331fb9332d6e9fdf338e67d  [@onsenwhale_42, @hedijung_86, @thebuyer_10, @...
2997  #3322441e6904f096c8a86ea5f263bf2a  [@doubleshot_151, @snova2000_280, @boboc_36, @...
2998  #8a990bbf90490dfb2a4c4ef8deffc136  [@tenbody_1389, @tenbody_1545, @illuhee_8, @dl..

In [13]:
f = open("/content/drive/MyDrive/MacBook/recommend.txt", 'w')

for i in range(len(dev_users_data)):
    f.write('%s %s\n' % (dev_users_data.iloc[i,0], ' '.join(dev_users_data.iloc[i,1])))

f.close()